In [ ]:
# download the weaviate client
%pip install -U weaviate-client

In [ ]:
import weaviate, os
from weaviate.config import AdditionalConfig, Timeout
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve environment variables
CLUSTER_URL = os.getenv("CLUSTER_URL")
API_KEY = os.getenv("API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

# Connect to Weaviate
client = weaviate.connect_to_local(
	auth_credentials=weaviate.auth.AuthApiKey("<API_KEY<>"),
    port=8087,
    skip_init_checks=True,
	headers={
		"X-OpenAI-Api-Key": OPENAI_API_KEY,
		"X-Cohere-Api-Key": COHERE_API_KEY
	},
	additional_config=AdditionalConfig(
		timeout=Timeout(init=30, query=60, insert=120)
			)
)

ready = client.is_ready()
server_version = client.get_meta()["version"]
client_version = weaviate.__version__

live = client.is_live()
connected = client.is_connected()

print(f"Weaviate client is ready: {ready}")
print(f"Weaviate Client Version: {client_version}")
print(f"Weaviate Server Version: {server_version}")
print(f"live: {live}")
print(f"Connected: {connected}")

In [ ]:
# Check existing users
existing_users = client.users.db.list_all()
print(f"Existing users: {existing_users}")

In [ ]:
# Check if role exist
print(f"<ROLE_NAME> role exists: {client.roles.exists(role_name='<ROLE_NAME>')}")

# List all roles
all_roles = client.roles.list_all()
print(f"All roles in system: {list(all_roles.keys())}")

In [ ]:
# Create comprehensive roles with ALL available permissions
from weaviate.classes.rbac import Permissions, RoleScope

# 1. SUPER ADMIN Role (Everything including User Management)
super_admin_permissions = [
    # Role Management (Full Control)
    Permissions.roles(
        role="*",
        create=True,
        read=True,
        update=True,
        delete=True,
        scope=RoleScope.ALL
    ),
    # User Management (Full Control)
    Permissions.users(
        user="*",
        create=True,
        read=True,
        update=True,
        delete=True,
        assign_and_revoke=True
    ),
    # Collections (Full Control)
    Permissions.collections(
        collection="*",
        create_collection=True,
        read_config=True,
        update_config=True,
        delete_collection=True
    ),
    # Tenants (Full Control)
    Permissions.tenants(
        collection="*",
        tenant="*",
        create=True,
        read=True,
        update=True,
        delete=True
    ),
    # Data Objects (Full CRUD)
    Permissions.data(
        collection="*",
        tenant="*",
        create=True,
        read=True,
        update=True,
        delete=True
    ),
    # Backups (Full Control)
    Permissions.backup(
        collection="*",
        manage=True
    ),
    # Cluster Data Access
    Permissions.cluster(read=True),
    # Node Data Access (Verbose)
    Permissions.Nodes.verbose(
        collection="*",
        read=True
    )
]

super_admin_result = client.roles.create(role_name="SuperAdmin", permissions=super_admin_permissions)

# 2. ADMIN Role (No Role/User Management)
admin_permissions = [
    # Collections (Full Control)
    Permissions.collections(
        collection="*",
        create_collection=True,
        read_config=True,
        update_config=True,
        delete_collection=True
    ),
    # Tenants (Full Control)
    Permissions.tenants(
        collection="*",
        tenant="*",
        create=True,
        read=True,
        update=True,
        delete=True
    ),
    # Data Objects (Full CRUD)
    Permissions.data(
        collection="*",
        tenant="*",
        create=True,
        read=True,
        update=True,
        delete=True
    ),
    # Backups (Full Control)
    Permissions.backup(
        collection="*",
        manage=True
    ),
    # Cluster Data Access
    Permissions.cluster(read=True),
    # Node Data Access (Verbose)
    Permissions.Nodes.verbose(
        collection="*",
        read=True
    )
]

admin_result = client.roles.create(role_name="Admin", permissions=admin_permissions)

# 3. EDITOR Role (Data + Limited Collection Management)
editor_permissions = [
    # Collections (Read/Update only)
    Permissions.collections(
        collection="*",
        read_config=True,
        update_config=True
    ),
    # Tenants (CRUD)
    Permissions.tenants(
        collection="*",
        tenant="*",
        create=True,
        read=True,
        update=True,
        delete=True
    ),
    # Data Objects (Full CRUD)
    Permissions.data(
        collection="*",
        tenant="*",
        create=True,
        read=True,
        update=True,
        delete=True
    ),
    # Node Data Access (Minimal)
    Permissions.Nodes.minimal(read=True)
]

editor_result = client.roles.create(role_name="Editor", permissions=editor_permissions)

# 4. READER Role (Read-only)
reader_permissions = [
    # Collections (Read only)
    Permissions.collections(
        collection="*",
        read_config=True
    ),
    # Tenants (Read only)
    Permissions.tenants(
        collection="*",
        tenant="*",
        read=True
    ),
    # Data Objects (Read only)
    Permissions.data(
        collection="*",
        tenant="*",
        read=True
    ),
    # Node Data Access (Minimal)
    Permissions.Nodes.minimal(read=True)
]

reader_result = client.roles.create(role_name="Reader", permissions=reader_permissions)

# 5. BACKUP MANAGER Role (Backup operations only)
backup_manager_permissions = [
    # Collections (Read only - needed to see what to backup)
    Permissions.collections(
        collection="*",
        read_config=True
    ),
    # Backups (Full Control)
    Permissions.backup(
        collection="*",
        manage=True
    ),
    # Cluster Data Access (needed for backup operations)
    Permissions.cluster(read=True)
]

backup_manager_result = client.roles.create(role_name="BackupManager", permissions=backup_manager_permissions)

# 6. MONITOR Role (Cluster/Node monitoring only)
monitor_permissions = [
    # Cluster Data Access
    Permissions.cluster(read=True),
    # Node Data Access (Verbose)
    Permissions.Nodes.verbose(
        collection="*",
        read=True
    )
]

monitor_result = client.roles.create(role_name="Monitor", permissions=monitor_permissions)

# 7. USER MANAGER Role (User management only)
user_manager_permissions = [
    # User Management (Full Control)
    Permissions.users(
        user="*",
        create=True,
        read=True,
        update=True,
        delete=True,
        assign_and_revoke=True
    ),
    # Role Management (Read only - to see what roles exist)
    Permissions.roles(
        role="*",
        read=True,
        scope=RoleScope.MATCH
    )
]

user_manager_result = client.roles.create(role_name="UserManager", permissions=user_manager_permissions)

print("All comprehensive roles created successfully!")
print(f"Roles created: SuperAdmin, Admin, Editor, Reader, BackupManager, Monitor, UserManager")

# Verify all roles exist
all_roles = client.roles.list_all()
print(f"All roles in system: {list(all_roles.keys())}")

In [ ]:
# Create 10 Random users manually
user_bob = client.users.db.create(user_id="Bob")
user_andre = client.users.db.create(user_id="Andre") 
user_joe = client.users.db.create(user_id="Joe")
user_mohamed = client.users.db.create(user_id="Mohamed")
user_dan = client.users.db.create(user_id="Dan")
user_alice = client.users.db.create(user_id="Alice")
user_charlie = client.users.db.create(user_id="Charlie")
user_david = client.users.db.create(user_id="David")
user_emma = client.users.db.create(user_id="Emma")
user_frank = client.users.db.create(user_id="Frank")

print(f"Created Bob with API key: {user_bob}")
print(f"Created Andre with API key: {user_andre}")
print(f"Created Joe with API key: {user_joe}")
print(f"Created Mohamed with API key: {user_mohamed}")
print(f"Created Dan with API key: {user_dan}")
print(f"Created Alice with API key: {user_alice}")
print(f"Created Charlie with API key: {user_charlie}")
print(f"Created David with API key: {user_david}")
print(f"Created Emma with API key: {user_emma}")
print(f"Created Frank with API key: {user_frank}")

# Assign roles manually
client.users.db.assign_roles(user_id="Bob", role_names=["SuperAdmin"])
client.users.db.assign_roles(user_id="Andre", role_names=["Admin"])
client.users.db.assign_roles(user_id="Joe", role_names=["Editor"])
client.users.db.assign_roles(user_id="Mohamed", role_names=["Reader"])
client.users.db.assign_roles(user_id="Dan", role_names=["BackupManager"])
client.users.db.assign_roles(user_id="Alice", role_names=["Monitor"])
client.users.db.assign_roles(user_id="Charlie", role_names=["UserManager"])
client.users.db.assign_roles(user_id="David", role_names=["Reader"])
client.users.db.assign_roles(user_id="Emma", role_names=["Editor"])
client.users.db.assign_roles(user_id="Frank", role_names=["BackupManager"])

print("All users created and roles assigned manually!")

In [ ]:
# Cleanup script to remove all custom roles and database users
# 1. Delete all custom roles (keep predefined ones: root, viewer)
all_roles = client.roles.list_all()
predefined_roles = ['root', 'viewer', 'admin']
custom_roles = [role for role in all_roles.keys() if role not in predefined_roles]

for role_name in custom_roles:
    try:
        client.roles.delete(role_name)
        print(f"✅ Deleted role: {role_name}")
    except Exception as e:
        print(f"❌ Failed to delete role {role_name}: {e}")

# 2. Delete all database users (not env users)
all_users = client.users.db.list_all()
print(f"Found {len(all_users)} users")

for user in all_users:
    # Check if it's a database user (not environment user)
    if user.user_type.value == 'db_user':  # user_type is an enum
        try:
            client.users.db.delete(user_id=user.user_id)
            print(f"✅ Deleted user: {user.user_id}")
        except Exception as e:
            print(f"❌ Failed to delete user {user.user_id}: {e}")
    else:
        print(f"⏭️  Skipping user: {user.user_id} (type: {user.user_type.value})")

# 3. Verify cleanup
print("\n📋 Remaining roles:")
remaining_roles = client.roles.list_all()
print(list(remaining_roles.keys()))

# Verify remaining users
print("\n📋 Remaining users:")
remaining_users = client.users.db.list_all()
for user in remaining_users:
    print(f"- {user.user_id} (type: {user.user_type.value})")

print(f"\n🧹 Cleanup complete! {len(remaining_users)} users remaining.")
print("\n🧹 Cleanup complete! Ready for fresh start.")

In [ ]:
# Role-Based Access Control (RBAC) Analysis Script
import pandas as pd
from typing import Dict, List, Any

def list_all_users():
    """Method 1: List all users in table format"""
    all_users = client.users.db.list_all()
    
    users_data = []
    for user in all_users:
        users_data.append({
            'User ID': user.user_id,
            'User Type': user.user_type.value,
            'Active': user.active,
            'Assigned Roles': ', '.join(user.role_names) if user.role_names else 'None'
        })
    
    df_users = pd.DataFrame(users_data)
    print("📋 ALL USERS")
    print("=" * 80)
    print(df_users.to_string(index=False))
    print(f"\nTotal Users: {len(df_users)}")
    return df_users

def list_all_roles():
    """Method 2: List all roles in table format"""
    all_roles = client.roles.list_all()
    
    roles_data = []
    for role_name, role_obj in all_roles.items():
        # Count total permissions
        total_permissions = 0
        permission_types = []
        
        if hasattr(role_obj, 'roles_permissions') and role_obj.roles_permissions:
            total_permissions += len(role_obj.roles_permissions)
            permission_types.append('Role Management')
        if hasattr(role_obj, 'users_permissions') and role_obj.users_permissions:
            total_permissions += len(role_obj.users_permissions)
            permission_types.append('User Management')
        if hasattr(role_obj, 'collections_permissions') and role_obj.collections_permissions:
            total_permissions += len(role_obj.collections_permissions)
            permission_types.append('Collections')
        if hasattr(role_obj, 'tenants_permissions') and role_obj.tenants_permissions:
            total_permissions += len(role_obj.tenants_permissions)
            permission_types.append('Tenants')
        if hasattr(role_obj, 'data_permissions') and role_obj.data_permissions:
            total_permissions += len(role_obj.data_permissions)
            permission_types.append('Data Objects')
        if hasattr(role_obj, 'backups_permissions') and role_obj.backups_permissions:
            total_permissions += len(role_obj.backups_permissions)
            permission_types.append('Backups')
        if hasattr(role_obj, 'cluster_permissions') and role_obj.cluster_permissions:
            total_permissions += len(role_obj.cluster_permissions)
            permission_types.append('Cluster')
        if hasattr(role_obj, 'nodes_permissions') and role_obj.nodes_permissions:
            total_permissions += len(role_obj.nodes_permissions)
            permission_types.append('Nodes')
        
        roles_data.append({
            'Role Name': role_name,
            'Permission Count': total_permissions,
            'Permission Types': ', '.join(permission_types) if permission_types else 'None'
        })
    
    df_roles = pd.DataFrame(roles_data)
    print("\n🎭 ALL ROLES")
    print("=" * 80)
    print(df_roles.to_string(index=False))
    print(f"\nTotal Roles: {len(df_roles)}")
    return df_roles

def list_all_permissions():
    """Method 3: List all permissions and associated permissions in table format"""
    all_roles = client.roles.list_all()
    
    permissions_data = []
    
    for role_name, role_obj in all_roles.items():
        # Role Management Permissions
        if hasattr(role_obj, 'roles_permissions') and role_obj.roles_permissions:
            for perm in role_obj.roles_permissions:
                actions = [action.value for action in perm.actions] if hasattr(perm, 'actions') else []
                permissions_data.append({
                    'Role Name': role_name,
                    'Permission Type': 'Role Management',
                    'Resource Filter': getattr(perm, 'role', '*'),
                    'Actions': ', '.join(actions),
                    'Scope': getattr(perm, 'scope', 'N/A')
                })
        
        # User Management Permissions
        if hasattr(role_obj, 'users_permissions') and role_obj.users_permissions:
            for perm in role_obj.users_permissions:
                actions = [action.value for action in perm.actions] if hasattr(perm, 'actions') else []
                permissions_data.append({
                    'Role Name': role_name,
                    'Permission Type': 'User Management',
                    'Resource Filter': getattr(perm, 'user', '*'),
                    'Actions': ', '.join(actions),
                    'Scope': 'N/A'
                })
        
        # Collections Permissions
        if hasattr(role_obj, 'collections_permissions') and role_obj.collections_permissions:
            for perm in role_obj.collections_permissions:
                actions = [action.value for action in perm.actions] if hasattr(perm, 'actions') else []
                permissions_data.append({
                    'Role Name': role_name,
                    'Permission Type': 'Collections',
                    'Resource Filter': getattr(perm, 'collection', '*'),
                    'Actions': ', '.join(actions),
                    'Scope': 'N/A'
                })
        
        # Tenants Permissions
        if hasattr(role_obj, 'tenants_permissions') and role_obj.tenants_permissions:
            for perm in role_obj.tenants_permissions:
                actions = [action.value for action in perm.actions] if hasattr(perm, 'actions') else []
                resource_filter = f"Collection: {getattr(perm, 'collection', '*')}, Tenant: {getattr(perm, 'tenant', '*')}"
                permissions_data.append({
                    'Role Name': role_name,
                    'Permission Type': 'Tenants',
                    'Resource Filter': resource_filter,
                    'Actions': ', '.join(actions),
                    'Scope': 'N/A'
                })
        
        # Data Objects Permissions
        if hasattr(role_obj, 'data_permissions') and role_obj.data_permissions:
            for perm in role_obj.data_permissions:
                actions = [action.value for action in perm.actions] if hasattr(perm, 'actions') else []
                resource_filter = f"Collection: {getattr(perm, 'collection', '*')}, Tenant: {getattr(perm, 'tenant', '*')}"
                permissions_data.append({
                    'Role Name': role_name,
                    'Permission Type': 'Data Objects',
                    'Resource Filter': resource_filter,
                    'Actions': ', '.join(actions),
                    'Scope': 'N/A'
                })
        
        # Backups Permissions
        if hasattr(role_obj, 'backups_permissions') and role_obj.backups_permissions:
            for perm in role_obj.backups_permissions:
                actions = [action.value for action in perm.actions] if hasattr(perm, 'actions') else []
                permissions_data.append({
                    'Role Name': role_name,
                    'Permission Type': 'Backups',
                    'Resource Filter': getattr(perm, 'collection', '*'),
                    'Actions': ', '.join(actions),
                    'Scope': 'N/A'
                })
        
        # Cluster Permissions
        if hasattr(role_obj, 'cluster_permissions') and role_obj.cluster_permissions:
            for perm in role_obj.cluster_permissions:
                actions = [action.value for action in perm.actions] if hasattr(perm, 'actions') else []
                permissions_data.append({
                    'Role Name': role_name,
                    'Permission Type': 'Cluster',
                    'Resource Filter': 'N/A',
                    'Actions': ', '.join(actions),
                    'Scope': 'N/A'
                })
        
        # Nodes Permissions
        if hasattr(role_obj, 'nodes_permissions') and role_obj.nodes_permissions:
            for perm in role_obj.nodes_permissions:
                actions = [action.value for action in perm.actions] if hasattr(perm, 'actions') else []
                verbosity = getattr(perm, 'verbosity', 'unknown')
                collection_filter = getattr(perm, 'collection', '*') if verbosity == 'verbose' else 'All'
                permissions_data.append({
                    'Role Name': role_name,
                    'Permission Type': f'Nodes ({verbosity})',
                    'Resource Filter': collection_filter,
                    'Actions': ', '.join(actions),
                    'Scope': 'N/A'
                })
    
    df_permissions = pd.DataFrame(permissions_data)
    print("\n🔐 ALL PERMISSIONS")
    print("=" * 120)
    print(df_permissions.to_string(index=False))
    print(f"\nTotal Permission Entries: {len(df_permissions)}")
    return df_permissions

def list_users_roles_permissions_combined():
    """Method 4: Combined view - users, roles, and permissions"""
    all_users = client.users.db.list_all()
    all_roles = client.roles.list_all()
    
    combined_data = []
    
    for user in all_users:
        if user.role_names:
            for role_name in user.role_names:
                if role_name in all_roles:
                    role_obj = all_roles[role_name]
                    
                    # Get permission summary for this role
                    permission_summary = []
                    
                    if hasattr(role_obj, 'roles_permissions') and role_obj.roles_permissions:
                        permission_summary.append('Role Management')
                    if hasattr(role_obj, 'users_permissions') and role_obj.users_permissions:
                        permission_summary.append('User Management')
                    if hasattr(role_obj, 'collections_permissions') and role_obj.collections_permissions:
                        permission_summary.append('Collections')
                    if hasattr(role_obj, 'tenants_permissions') and role_obj.tenants_permissions:
                        permission_summary.append('Tenants')
                    if hasattr(role_obj, 'data_permissions') and role_obj.data_permissions:
                        permission_summary.append('Data Objects')
                    if hasattr(role_obj, 'backups_permissions') and role_obj.backups_permissions:
                        permission_summary.append('Backups')
                    if hasattr(role_obj, 'cluster_permissions') and role_obj.cluster_permissions:
                        permission_summary.append('Cluster')
                    if hasattr(role_obj, 'nodes_permissions') and role_obj.nodes_permissions:
                        permission_summary.append('Nodes')
                    
                    combined_data.append({
                        'User ID': user.user_id,
                        'User Type': user.user_type.value,
                        'Role Name': role_name,
                        'Active': user.active,
                        'Permission Areas': ', '.join(permission_summary) if permission_summary else 'None'
                    })
                else:
                    # Role exists in user but not found in roles list
                    combined_data.append({
                        'User ID': user.user_id,
                        'User Type': user.user_type.value,
                        'Role Name': role_name,
                        'Active': user.active,
                        'Permission Areas': 'Role not found'
                    })
        else:
            # User has no roles
            combined_data.append({
                'User ID': user.user_id,
                'User Type': user.user_type.value,
                'Role Name': 'None',
                'Active': user.active,
                'Permission Areas': 'No permissions'
            })
    
    df_combined = pd.DataFrame(combined_data)
    print("\n👥 USERS + ROLES + PERMISSIONS COMBINED")
    print("=" * 120)
    print(df_combined.to_string(index=False))
    print(f"\nTotal User-Role Assignments: {len(df_combined)}")
    return df_combined

# Execute all methods
print("RBAC ANALYSIS REPORT")
print("=" * 80)

df_users = list_all_users()
df_roles = list_all_roles()
df_permissions = list_all_permissions()
df_combined = list_users_roles_permissions_combined()